In [1]:
from __future__ import division, print_function, absolute_import
from past.builtins import basestring

import os
from decimal import Decimal
from traceback import print_exc

import pandas as pd
import pandas_profiling

from pug.nlp.util import dict2obj

from twip.constant import DATA_PATH

In [5]:
import matplotlib
from IPython.display import display, HTML 
%matplotlib inline
np = pd.np
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_rows', 6)
pd.set_option('display.max_columns', 250)

In [18]:
df = pd.read_csv(os.path.join(DATA_PATH, 'cleaned_tweets.csv.gz'), low_memory=False)
rawlen = len(df)
df.drop_duplicates('id_str', keep='last', inplace=True)
rawlen - len(df)

10308


In [19]:
df

,Unnamed: 0,id,coordinates_coordinates,coordinates_type,created_at,entities_hashtags,entities_media,entities_symbols,entities_urls,entities_user_mentions,favorite_count,favorited,geo_coordinates,geo_type,id_str,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,is_quote_status,lang,lat,lon,metadata_iso_language_code,metadata_result_type,place_bounding_box_coordinates,place_bounding_box_type,place_contained_within,place_country,place_country_code,place_full_name,place_id,place_name,place_place_type,place_url,possibly_sensitive,quoted_status_created_at,quoted_status_entities_hashtags,quoted_status_entities_media,quoted_status_entities_symbols,quoted_status_entities_urls,quoted_status_entities_user_mentions,quoted_status_favorite_count,quoted_status_favorited,quoted_status_id,quoted_status_id_str,quoted_status_is_quote_status,quoted_status_lang,quoted_status_metadata_iso_language_code,quoted_status_metadata_result_type,quoted_status_possibly_sensitive,quoted_status_retweet_count,quoted_status_retweeted,quoted_status_source,quoted_status_text,quoted_status_truncated,quoted_status_user_contributors_enabled,quoted_status_user_created_at,quoted_status_user_default_profile,quoted_status_user_default_profile_image,quoted_status_user_description,quoted_status_user_entities_description_urls,quoted_status_user_entities_url_urls,quoted_status_user_favourites_count,quoted_status_user_followers_count,quoted_status_user_friends_count,quoted_status_user_geo_enabled,quoted_status_user_has_extended_profile,quoted_status_user_id,quoted_status_user_id_str,quoted_status_user_is_translation_enabled,quoted_status_user_is_translator,quoted_status_user_lang,quoted_status_user_listed_count,quoted_status_user_location,quoted_status_user_name,quoted_status_user_profile_background_color,quoted_status_user_profile_background_image_url,quoted_status_user_profile_background_image_url_https,quoted_status_user_profile_background_tile,quoted_status_user_profile_banner_url,quoted_status_user_profile_image_url,quoted_status_user_profile_image_url_https,quoted_status_user_profile_link_color,quoted_status_user_profile_sidebar_border_color,quoted_status_user_profile_sidebar_fill_color,quoted_status_user_profile_text_color,quoted_status_user_profile_use_background_image,quoted_status_user_protected,quoted_status_user_screen_name,quoted_status_user_statuses_count,quoted_status_user_time_zone,quoted_status_user_url,quoted_status_user_utc_offset,quoted_status_user_verified,quoted_status_id.1,quoted_status_id_str.1,retweet_count,retweeted,retweeted_status_created_at,retweeted_status_entities_hashtags,retweeted_status_entities_media,retweeted_status_entities_symbols,retweeted_status_entities_urls,retweeted_status_entities_user_mentions,retweeted_status_favorite_count,retweeted_status_favorited,retweeted_status_id,retweeted_status_id_str,retweeted_status_in_reply_to_screen_name,retweeted_status_in_reply_to_status_id,retweeted_status_in_reply_to_status_id_str,retweeted_status_in_reply_to_user_id,retweeted_status_in_reply_to_user_id_str,retweeted_status_is_quote_status,retweeted_status_lang,retweeted_status_metadata_iso_language_code,retweeted_status_metadata_result_type,retweeted_status_place_bounding_box_coordinates,retweeted_status_place_bounding_box_type,retweeted_status_place_contained_within,retweeted_status_place_country,retweeted_status_place_country_code,retweeted_status_place_full_name,...,retweeted_status_quoted_status_user_default_profile,retweeted_status_quoted_status_user_default_profile_image,retweeted_status_quoted_status_user_description,retweeted_status_quoted_status_user_entities_description_urls,retweeted_status_quoted_status_user_entities_url_urls,retweeted_status_quoted_status_user_favourites_count,retweeted_status_quoted_status_user_followers_count,retweeted_status_quoted_status_user_friends_count,retweeted_status_quoted_status_user_geo_enabled,retweeted_status_quoted_status_user_has_exten

In [20]:
mask = np.array([bool('date' in c) for c in df.columns])
df.columns[mask]

Index([], dtype='object')

In [21]:
df.columns[np.array([bool('time' in c) for c in df.columns])]

Index([u'quoted_status_user_time_zone',
       u'retweeted_status_quoted_status_user_time_zone',
       u'retweeted_status_user_time_zone', u'user_time_zone'],
      dtype='object')

Remember any date or time columns from Dan's tutorial?

In [22]:
mask = np.array([c.endswith('_at') for c in df.columns])
df.columns[mask]

Index([u'created_at', u'quoted_status_created_at',
       u'quoted_status_user_created_at', u'retweeted_status_created_at',
       u'retweeted_status_quoted_status_created_at',
       u'retweeted_status_quoted_status_user_created_at',
       u'retweeted_status_user_created_at', u'user_created_at'],
      dtype='object')

In [27]:
dates = df[df.columns[mask]]
dates

,created_at,quoted_status_created_at,quoted_status_user_created_at,retweeted_status_created_at,retweeted_status_quoted_status_created_at,retweeted_status_quoted_status_user_created_at,retweeted_status_user_created_at,user_created_at
87,Sun Apr 24 16:39:11 +0000 2016,NaN,NaN,NaN,NaN,NaN,NaN,Fri Jan 02 23:58:29 +0000 2015
88,Sun Apr 24 16:39:08 +0000 2016,NaN,NaN,NaN,NaN,NaN,NaN,Fri Oct 30 13:32:51 +0000 2009
90,Sun Apr 24 16:38:42 +0000 2016,NaN,NaN,NaN,NaN,NaN,NaN,Tue Apr 30 05:44:27 +0000 2013
...,...,...,...,...,...,...,...,...
193375,Sun Apr 24 16:35:36 +0000 2016,NaN,NaN,Sun Apr 24 16:07:50 +0000 2016,NaN,NaN,Wed Aug 08 20:44:42 +0000 2012,Mon Aug 17 22:36:39 +0000 2015
193376,Sun Apr 24 16:35:29 +0000 2016,NaN,NaN,NaN,NaN,NaN,NaN,Wed Oct 05 01:11:53 +0000 2011
193377,Sun Apr 24 16:35:26 +0000 2016,NaN,NaN,NaN,NaN,NaN,NaN,Sun May 22 03:29:30 +0000 2011


In [28]:
dates = pd.to_datetime(dates)

ValueError: to assemble mappings requires at least that [year, month, day] be specified: [day,month,year] is missing

In [29]:
dates = pd.DataFrame(index=df.index)
for col in df.columns[np.array([bool(c.endswith('_at')) for c in df.columns])]:
    print(col)
    dates[col] = pd.to_datetime(df[col])

created_at
quoted_status_created_at
quoted_status_user_created_at
retweeted_status_created_at
retweeted_status_quoted_status_created_at
retweeted_status_quoted_status_user_created_at
retweeted_status_user_created_at
user_created_at


In [30]:
dates

,created_at,quoted_status_created_at,quoted_status_user_created_at,retweeted_status_created_at,retweeted_status_quoted_status_created_at,retweeted_status_quoted_status_user_created_at,retweeted_status_user_created_at,user_created_at
87,2016-04-24 16:39:11,NaT,NaT,NaT,NaT,NaT,NaT,2015-01-02 23:58:29
88,2016-04-24 16:39:08,NaT,NaT,NaT,NaT,NaT,NaT,2009-10-30 13:32:51
90,2016-04-24 16:38:42,NaT,NaT,NaT,NaT,NaT,NaT,2013-04-30 05:44:27
...,...,...,...,...,...,...,...,...
193375,2016-04-24 16:35:36,NaT,NaT,2016-04-24 16:07:50,NaT,NaT,2012-08-08 20:44:42,2015-08-17 22:36:39
193376,2016-04-24 16:35:29,NaT,NaT,NaT,NaT,NaT,NaT,2011-10-05 01:11:53
193377,2016-04-24 16:35:26,NaT,NaT,NaT,NaT,NaT,NaT,2011-05-22 03:29:30


In [33]:
dates.to_csv(os.path.join(DATA_PATH, 'datetimes.csv.gz'), compression='gzip')

ls: cannot access 'DATA_PATH': No such file or directory


In [36]:
%ls -thal DATA_PATH
!ls -thal data

ls: cannot access 'DATA_PATH': No such file or directory
ls: cannot access 'data': No such file or directory


In [43]:
system("ls %s" % DATA_PATH)

['/bin/sh: 1: ls %s: not found']

In [45]:
ls -thal ../../data

total 3.2G
-rw-rw-r--  1 hobs hobs 2.4M May 27 21:15 datetimes.csv.gz
drwxrwxr-x  2 hobs hobs  12K May 27 21:13 ./
-rw-rw-r--  1 hobs hobs 8.5M May 27 20:20 text.csv.gz
-rw-rw-r--  1 hobs hobs 9.0M May 27 20:14 numbers.csv.gz
-rw-rw-r--  1 hobs hobs 2.5M May 27 19:41 timestamps.csv.gz
-rw-rw-r--  1 hobs hobs  82M May 27 11:56 cleaned_tweets.csv.gz
-rw-rw-r--  1 hobs hobs  82M May 27 11:40 cleaned_tweets2.csv.gz
-rw-rw-r--  1 hobs hobs 531M May 27 10:44 unclean_tweets.csv
-rw-rw-r--  1 hobs hobs 531M May 27 10:37 cleaned_tweets.csv
drwxrwxr-x 11 hobs hobs 4.0K May 27 09:03 ../
-rw-rw-r--  1 hobs hobs 2.3K May 26 22:42 tweets20.txt
-rw-rw-r--  1 hobs hobs 3.6M May 26 22:42 tweets.zip
-rw-rw-r--  1 hobs hobs  20M May 26 22:42 data.csv.gz
-rw-rw-r--  1 hobs hobs  36K May 26 22:42 geo.svg
-rw-rw-r--  1 hobs hobs 544M May 26 07:45 all_tweets.csv
-rw-r--r--  1 hobs hobs 921K May 25 14:39 2016-05-23 02:51:38.942055.json
-rw-r--r--  1 hobs hobs 941K May 25 14:39 2016-05-23 03:07:54.957982.json
